In [1]:
#Importing all required Modules

from sqlalchemy.engine import create_engine
import pandas as pd
import datetime
from sqlalchemy.dialects.oracle import DATE, VARCHAR2, FLOAT, NUMBER,TIMESTAMP
from sqlalchemy import text,String,types
from datetime import datetime

In [3]:
# Extracting data from Source

with source_engine.begin():
    try:
        query = ('select distinct Appointment_Provider_Name from AceExtraction.Medrite_input_table;')
        provider_names_df = pd.read_sql(query, source_engine)
        provider_names = list(provider_names_df['Appointment_Provider_Name'])
    except Exception as e:
        print('Error: ',e)

#provider_names_df
#provider_names

In [17]:
#creating empty dataframe

service_provider_dim_df = pd.DataFrame(columns=['service_provider_id','service_provider_full_name',
                                             'service_provider_specialization',
                                             'service_provider_job_role','service_provider_gender',
                                             'created_date'])
#service_provider_dim_df

In [18]:
#Loading the data into dataframe

service_provider_dim_df['service_provider_full_name'] = provider_names

In [19]:
#Defining the data types

data_types = {'service_provider_id':types.Integer,
                            'service_provider_full_name': types.String(255),
                            'service_provider_specialization': types.String(255),
                            'service_provider_job_role':types.String(255),
                            'service_provider_gender':types.String(255),
                            'created_date':types.String(10)}

In [20]:
#Loading the data into Staging DB

try:
 service_provider_dim_df.to_sql('service_provider_dim',staging_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into Staging_DB')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into Staging_DB


##ETL from staging to DataWarehouse

In [21]:
#Extracting data from staging DB

with staging_engine.begin():
 try:
     query = ("select distinct service_provider_full_name from staging_source.service_provider_dim;")
     staging_service_provider_full_name_df = pd.read_sql_query(query, staging_engine)
     staging_service_provider_full_name_df.dropna(subset=['service_provider_full_name'], inplace=True)
     staging_service_provider_full_name = list(staging_service_provider_full_name_df['service_provider_full_name'])
 except Exception as e:
     print("An error occurred:", e)

     
#staging_service_provider_full_name_df
#staging_service_provider_full_name

In [22]:
#creating empty dataframe

service_provider_dim_df = pd.DataFrame(columns=['service_provider_id','service_provider_first_name',
                                             'service_provider_last_name','service_provider_specialization',
                                             'service_provider_job_role','service_provider_gender',
                                             'created_date'])
#service_provider_dim_df

In [23]:
# Determining the current maximum ID in the DataFrame, 

current_max_id = service_provider_dim_df['service_provider_id'].max() if not service_provider_dim_df.empty else 0
# Using pandas Series to generate IDs
new_ids = pd.Series(range(current_max_id + 1, current_max_id + 1 + len(staging_service_provider_full_name)))

In [24]:
#Providing the full forms of job roles

job_role_mapping = {
    'PT': 'Physical Therapist',
    'MD': 'Doctor of Medicine',
    'FNP': 'Family Nurse Practitioner',
    'APRN': 'Advanced Practice Registered Nurse',
    'PA': 'Physician Assistant',
    'NP': 'Nurse Practitioner',
    'DO': 'Doctor of Osteopathic Medicine',
    'Provider': 'Provider',
    'PA-C': 'Physician Assistant-Certified',
    'PMHNP': 'Psychiatric Mental Health Nurse Practitioner',
    'FNP-BC': 'Family Nurse Practitioner-Board Certified',
    'FNP-C': 'Family Nurse Practitioner-Certified',
    'LPC': 'Licensed Professional Counselor'
}


In [25]:
#Transformations

names = staging_service_provider_full_name_df['service_provider_full_name'].str.split(",",n=1,expand=True)
first_name = names[1]
last_name = names[0].str.split(" ",n=1,expand=True)[0]
job_role = names[0].str.split(" ", n=1, expand=True)[1].apply(lambda x: job_role_mapping.get(x, x))
job_role = job_role.fillna('Job Role Not Available')


In [26]:
new_records = pd.DataFrame({'service_provider_id':new_ids,
                            'service_provider_first_name': first_name,
                            'service_provider_last_name': last_name,
                            'service_provider_specialization': 'Unknown',
                            'service_provider_job_role':job_role,
                            'service_provider_gender':'Unknown',
                            'created_date':datetime.now().strftime('%d-%m-%y')})

In [27]:
data_types = {'service_provider_id':types.Integer,
                            'service_provider_first_name': types.String(255),
                            'service_provider_last_name': types.String(255),
                            'service_provider_specialization': types.String(255),
                            'service_provider_job_role':types.String(255),
                            'service_provider_gender':types.String(255),
                            'created_date':types.Date}

In [28]:
#Loading the data into Datawarehouse

try:
    new_records.to_sql('service_provider_dim',datawarehouse_engine, if_exists='append', index=False, dtype=data_types)
    print('successfully loaded into Data Warehouse')
except Exception as e:
        print("An error occurred:", e)

successfully loaded into Data Warehouse
